how does python access attributes in an object

In [1]:
class Person:
    pass

attribute's value can be in any number of places:
- instance dict
- class attribute
- descriptor
- in a parent class

`__getattribute__` bound method is called first, when we call person.name for example

we can override it by implementing `__getattribute__(self, name)`

usually we override `__getattr__` and not `__getattribute__`

`__getattr__` method is called by Python if `__getattribute__` cannot find the requested attribute and raises AttributeError

`__getattr__`'s default implementation only raises AttributeError

if we call `__getattribute__` ourselves and it fails, `__getattr__` doesn't get called by itself. we may want to call it specifically

_see lookup resolution in download folder_

`__getattribute__` and `__getattr__` are instance methods.

how do we override class attribute access? using metaclass.

In [2]:
p = Person()
try:
    p.name
except AttributeError as ex:
    print(ex)

'Person' object has no attribute 'name'


let's override getattr method. `__getattr__` is called when `__getattribute__` returns an error

In [3]:
class Person:
    def __getattr__(self, name):
        print(f"__getattribute__ did not find {name}")
        return "not found!"

In [4]:
p = Person()

p.name


__getattribute__ did not find name


'not found!'

we should avoid infinite recursions

In [5]:
class Person:
    def __getattr__(self, name):
        print(f"__getattribute__ did not find {name}")
        alt_name = "_" + name
        if getattr(self, alt_name, None) is None:
            return getattr(self, alt_name)
        else:
            raise AttributeError(f"could not find {name} or {alt_name}")

In [7]:
p = Person()

p.age


__getattribute__ did not find age
__getattribute__ did not find _age
__getattribute__ did not find __age
__getattribute__ did not find ___age
__getattribute__ did not find ____age
__getattribute__ did not find _____age
__getattribute__ did not find ______age
__getattribute__ did not find _______age
__getattribute__ did not find ________age
__getattribute__ did not find _________age
__getattribute__ did not find __________age
__getattribute__ did not find ___________age
__getattribute__ did not find ____________age
__getattribute__ did not find _____________age
__getattribute__ did not find ______________age
__getattribute__ did not find _______________age
__getattribute__ did not find ________________age
__getattribute__ did not find _________________age
__getattribute__ did not find __________________age
__getattribute__ did not find ___________________age
__getattribute__ did not find ____________________age
__getattribute__ did not find _____________________age
__getattribute__ did 

RecursionError: maximum recursion depth exceeded while calling a Python object

let's fix it

In [8]:
class Person:
    def __getattr__(self, name):
        alt_name = "_" + name
        print(f"__getattribute__ did not find {name}, trying {alt_name}")
        try:
            return super().__getattribute__(alt_name) # we call __getattribute__ directly, 
        #so python will not call  __getattr__
        except AttributeError:
            raise AttributeError(f"could not find {name} or {alt_name}")
        

In [9]:
p = Person()

p.age


__getattribute__ did not find age, trying _age


AttributeError: could not find age or _age

In [10]:
try:
    p.age
except AttributeError as err:
    print(err)

__getattribute__ did not find age, trying _age
could not find age or _age


without try-except:

In [11]:
class Person:
    def __getattr__(self, name):
        alt_name = "_" + name
        print(f"__getattribute__ did not find {name}, trying {alt_name}")
        
        return super().__getattribute__(alt_name) 
        

In [12]:
p = Person()

p.age
# not informative

__getattribute__ did not find age, trying _age


AttributeError: 'Person' object has no attribute '_age'

In [14]:
class Person:
    def __init__(self, age):
        self._age = age
    
    def __getattr__(self, name):
        alt_name = "_" + name
        print(f"__getattribute__ did not find {name}, trying {alt_name}")
        try:
            return super().__getattribute__(alt_name) # we call __getattribute__ directly, 
        #so python will not call  __getattr__
        except AttributeError:
            raise AttributeError(f"could not find {name} or {alt_name}")

In [15]:
p = Person(43)

p.age

__getattribute__ did not find age, trying _age


43

In [16]:
p._age

43

we may want to be able to access attributes of the class, even if they don't exist, like default dict

In [21]:
class DefaultClass:
    def __init__(self, attribute_default=None):
        self._attribute_default = attribute_default
        
    def __getattr__(self, name):
        print(f"{name} not found; creating it and setting it to default...")
        setattr(self, name, self._attribute_default)
        return self._attribute_default

In [22]:
d = DefaultClass("NotAvailable")
d.test

test not found; creating it and setting it to default...


'NotAvailable'

In [23]:
d.__dict__

{'_attribute_default': 'NotAvailable', 'test': 'NotAvailable'}

In [24]:
d.age

age not found; creating it and setting it to default...


'NotAvailable'

In [25]:
d.__dict__

{'_attribute_default': 'NotAvailable',
 'test': 'NotAvailable',
 'age': 'NotAvailable'}

In [26]:
d.age

'NotAvailable'

In [27]:
 d.age = 18

In [28]:
d.age

18

In [29]:
d.__dict__

{'_attribute_default': 'NotAvailable', 'test': 'NotAvailable', 'age': 18}

now we can inherit from this class to provide its functionality

In [30]:
class Person(DefaultClass):
    def __init__(self, name):
        super().__init__("Unavailable")
        self.name = name

In [31]:
p = Person("Alex")
p.name

'Alex'

In [32]:
p.age

age not found; creating it and setting it to default...


'Unavailable'

Another usecase: logging the fact, that nonexisting attribute was requested

In [33]:
class AttributeNotFoundLogger:
    def __getattr__(self, name):
        err_msg = f"{type(self).__name__} object has no attribute {name}."
        print(f"Log: {err_msg}")
        raise AttributeError(err_msg)

In [34]:
class Person(AttributeNotFoundLogger):
    def __init__(self, name):
        self.name = name

In [35]:
p = Person("Alex")

In [36]:
p.name

'Alex'

In [37]:
try:
    p.age  
except AttributeError as err:
    print(err)

Log: Person object has no attribute age.
Person object has no attribute age.


let's take a look at `__getattribute__`  method. we should be very careful with infinite recursion 

Example: we want to stop people from accessing `_attributes`

In [38]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    def __getattribute__(self, name):
        if name.startswith('_'):
            raise AttributeError(f"Forbidden access to {name}")
        return super().__getattribute__( name)

In [39]:
p = Person("Alex", 19)

In [40]:
p._name

AttributeError: Forbidden access to _name

but `__dict__` is unavailable too

In [41]:
p.__dict__

AttributeError: Forbidden access to __dict__

In [42]:
vars(p)

TypeError: vars() argument must have __dict__ attribute

let's fix it

In [43]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    def __getattribute__(self, name):
        if name.startswith('_') and not name.startswith('__'):
            raise AttributeError(f"Forbidden access to {name}")
        return super().__getattribute__(name)

In [44]:
p = Person("Alex", 19)

In [45]:
p._name

AttributeError: Forbidden access to _name

In [46]:
p.__dict__

{'_name': 'Alex', '_age': 19}

In [47]:
vars(p)

{'_name': 'Alex', '_age': 19}

but we still doesn't have access to name and age. let's work on this

this won't work:

In [48]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    def __getattribute__(self, name):
        if name.startswith('_') and not name.startswith('__'):
            raise AttributeError(f"Forbidden access to {name}")
        return super().__getattribute__(name)
    
    @property
    def name(self):
        return self._name
    @property
    def age(self):
        return self._age

In [49]:
p = Person("Alex", 19)
p.name

AttributeError: Forbidden access to _name

let's fix it

In [50]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    def __getattribute__(self, name):
        if name.startswith('_') and not name.startswith('__'):
            raise AttributeError(f"Forbidden access to {name}")
        return super().__getattribute__(name)
    
    @property
    def name(self):
        return super().__getattribute__('_name')
    @property
    def age(self):
        return super().__getattribute__('_age')


In [51]:
p = Person("Alex", 19)
p.name

'Alex'

In [52]:
p._name

AttributeError: Forbidden access to _name

let's mix it with DefaultClass

we need to change our defaultClass first

In [53]:
class DefaultClass:
    def __init__(self, attribute_default=None):
        self._attribute_default = attribute_default
        
    def __getattr__(self, name):
        print(f"{name} not found; creating it and setting it to default...")
        default_value = super().__getattribute__('_attribute_default')
        setattr(self, name, default_value)
        return default_value

In [55]:
class Person(DefaultClass):
    def __init__(self, name, age):
        super().__init__("NotAvailable")
        if name is not None:
            self._name = name
        if age is not None:    
            self._age = age
        
    def __getattribute__(self, name):
        if name.startswith('_') and not name.startswith('__'):
            raise AttributeError(f"Forbidden access to {name}")
        return super().__getattribute__(name)
    
    @property
    def name(self):
        return super().__getattribute__('_name')
    @property
    def age(self):
        return super().__getattribute__('_age')


In [56]:
p = Person("Python", 42)
p.name

'Python'

In [57]:
p.language

language not found; creating it and setting it to default...


'NotAvailable'

In [58]:
p.__dict__

{'_attribute_default': 'NotAvailable',
 '_name': 'Python',
 '_age': 42,
 'language': 'NotAvailable'}

what about class attributes? how can we override accessors for class attributes?

In [59]:
class MetaLogger(type):
    def __getattribute__(self, name):
        print("class __getattribute__ called...")
        return super().__getattribute__(name)
    def __getattr__(self, name):
        print("class __getattr__ called...")
        return "Not Found"

In [60]:
class Account(metaclass=MetaLogger):
    apr = 10

In [61]:
Account.apr

class __getattribute__ called...


10

In [62]:
Account.afrfr

class __getattribute__ called...
class __getattr__ called...


'Not Found'

it doesn't matter if we access callable or attribute, it still runs `__getattribute__`

In [63]:
class MyClass:
    def __getattribute__(self, name):
        print("class __getattribute__ called...")
        return super().__getattribute__(name)
    
    def __getattr__(self, name):
        print("class __getattr__ called...")
        raise AttributeError(f"{name} not found.")
        
    def say_hello(self):
        return 'hello'

In [64]:
m = MyClass()

In [65]:
m.say_hello()

class __getattribute__ called...


'hello'

In [66]:
m.other()

class __getattribute__ called...
class __getattr__ called...


AttributeError: other not found.